In [6]:
import os
import glob
import pandas as pd
import pdfplumber
import re
from IPython.display import display

# --- 1. SETUP: Define your data and output directories ---
BASE_DIR = os.getcwd() 
DATA_DIR = os.path.join(BASE_DIR, "data")
OUT_DIR = os.path.join(BASE_DIR, "output")

# Create directories if they don't exist
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print(f"Data will be read from: {DATA_DIR}")
print(f"Output will be saved to: {OUT_DIR}")

Data will be read from: c:\Users\saifi\GitHub\InsurancePremiumAnalyzer\data\SBI GENERAL INSURANCE\data
Output will be saved to: c:\Users\saifi\GitHub\InsurancePremiumAnalyzer\data\SBI GENERAL INSURANCE\output


In [7]:
# --- Helper function for cleaning numeric values ---
def clean_value(text):
    """Cleans a string to extract a numeric value, returning 0 on failure."""
    if text is None:
        return 0
    cleaned_text = re.sub(r'[^\d.-]', '', str(text))
    if not cleaned_text or cleaned_text == '-':
        return 0
    try:
        return int(float(cleaned_text))
    except (ValueError, TypeError):
        return 0

# --- THE NEW UNIVERSAL PARSER ---
def parse_universal_nl4(pdf_path):
    """
    Extracts data from FORM NL-4 by trying multiple strategies.
    It first attempts a simple header extraction, and if that fails,
    it tries a more complex multi-level header extraction.
    
    The `laparams` argument is added to pdfplumber.open to improve
    text layout analysis for complex PDFs, which can help avoid the
    issues hinted at by the "Cannot set gray" warnings.
    """
    provider = "Unknown"
    month_map = {'june': 'Q1', 'september': 'Q2', 'december': 'Q3', 'march': 'Q4'}
    
    try:
        # Added laparams to help with layout analysis on complex PDFs
        with pdfplumber.open(pdf_path, laparams={"line_margin": 0.5, "char_margin": 2.0}) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text(x_tolerance=2, layout=True) or ""
                if "FORM NL-4" not in page_text or "PREMIUM SCHEDULE" not in page_text:
                    continue

                # --- STEP 1: Extract Provider Name ---
                provider_match = re.search(r"Name\s+of\s+(?:the\s+)?Insurer\s*[:\s]+\s*(.+)", page_text, re.IGNORECASE)
                if provider_match:
                    provider = provider_match.group(1).strip()
                else: # Fallback for other formats
                    for line in page_text.splitlines()[:5]:
                        if "Insurance" in line and ("Ltd" in line or "Limited" in line):
                            provider = line.strip()
                            break

                # --- STEP 2: Extract Year and Quarter (Flexible Regex) ---
                year, quarter = None, None
                # Regex to find "For the Quarter September, 2024" OR "For the quarter ended 30th June 2024"
                date_match = re.search(r"For\s+the\s+Quarter\s+(?:Ended\s*)?(?:on\s|at\s)?(?:the\s)?\d{0,2}\w{0,2}\s*([A-Za-z]+)[, ]+\s*(\d{4})", page_text, re.IGNORECASE)
                if date_match:
                    month_str, year_str = date_match.group(1).lower(), date_match.group(2)
                    year = int(year_str)
                    quarter = month_map.get(month_str[:3])
                
                if not year or not quarter: continue

                # --- STEP 3: Extract Table ---
                table = page.extract_table({
                    "vertical_strategy": "lines", "horizontal_strategy": "text",
                    "text_x_tolerance": 2, "text_y_tolerance": 2,
                })
                if not table: continue

                # --- STRATEGY 1: Simple, Single-Row Header (like Aditya Birla) ---
                header_indices = {}
                for r in reversed(table):
                    row_str = " ".join(filter(None, [str(s).replace('\n', ' ') for s in r]))
                    if "Personal Accident" in row_str and "Travel" in row_str and "Health" in row_str:
                        for i, cell in enumerate(r):
                            if cell:
                                clean_cell = cell.replace('\n', ' ').strip()
                                if clean_cell == "Health": header_indices['health'] = i
                                elif clean_cell == "Personal Accident": header_indices['personal_accident'] = i
                                elif clean_cell == "Travel Insurance": header_indices['travel'] = i
                        if 'health' in header_indices and 'personal_accident' in header_indices:
                            break
                
                # If simple strategy works, find data and return
                if 'health' in header_indices:
                    for r in table:
                        if r and any(cell and "Gross Direct Premium" in str(cell) for cell in r):
                            return {
                                "provider": provider, "year": year, "quarter": quarter,
                                "health": clean_value(r[header_indices.get('health')]),
                                "personal_accident": clean_value(r[header_indices.get('personal_accident')]),
                                "travel": clean_value(r[header_indices.get('travel')]),
                                "total": clean_value(r[header_indices.get('total')]) or (clean_value(r[header_indices.get('health')]) + clean_value(r[header_indices.get('personal_accident')]) + clean_value(r[header_indices.get('travel')])),
                                "source_file": os.path.basename(pdf_path)
                            }
                
                # --- STRATEGY 2: Multi-Level Header (like SBI) ---
                h1_idx, h2_idx = None, None
                for i, row in enumerate(table):
                    row_str = " ".join(filter(None, [str(c).replace('\n', ' ') for c in row]))
                    if "Personal Accident" in row_str and "Health Insurance" in row_str:
                        if i + 1 < len(table):
                            next_row_str = " ".join(filter(None, [str(c).replace('\n', ' ') for c in table[i+1]]))
                            if "For the quarter" in next_row_str:
                                h1_idx, h2_idx = i, i + 1
                                break
                
                if h1_idx is not None:
                    header_map = {}
                    current_primary = ""
                    for j, cell in enumerate(table[h1_idx]):
                        if cell and cell.strip(): current_primary = cell.replace('\n', ' ').strip()
                        s_header = table[h2_idx][j].replace('\n', ' ').strip() if table[h2_idx][j] else ""
                        header_map[j] = (current_primary, s_header)
                    
                    col_indices = {}
                    for idx, (p_header, s_header) in header_map.items():
                        if "For the quarter" in s_header and str(year) in s_header:
                            if "Health" in p_header: col_indices['health'] = idx
                            elif "Personal Accident" in p_header: col_indices['personal_accident'] = idx
                            elif "Travel" in p_header: col_indices['travel'] = idx
                    
                    if 'health' in col_indices:
                        for r in table:
                             if r and any(cell and "Gross Direct Premium" in str(cell) for cell in r):
                                h_val = clean_value(r[col_indices.get('health')])
                                pa_val = clean_value(r[col_indices.get('personal_accident')])
                                t_val = clean_value(r[col_indices.get('travel')])
                                return {
                                    "provider": provider, "year": year, "quarter": quarter,
                                    "health": h_val, "personal_accident": pa_val, "travel": t_val,
                                    "total": h_val + pa_val + t_val,
                                    "source_file": os.path.basename(pdf_path)
                                }
                break # Processed the correct page, move to next PDF
    except Exception as e:
        print(f"  -> An error occurred while processing {os.path.basename(pdf_path)}: {e}")
    
    return None # Return None if all strategies fail


In [ ]:
# --- 3. PROCESSING LOOP: Extract data from all PDFs ---
records = []
failed_files = []

pdf_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.pdf")))

if not pdf_files:
    print(f"❌ No PDF files found in '{DATA_DIR}'.")
    print("Please make sure your PDF files are in the 'data' folder.")
else:
    for pdf_file in pdf_files:
        print(f"📄 Processing: {os.path.basename(pdf_file)}")
        # Use the NEW universal parsing function
        result = parse_universal_nl4(pdf_file)
        if result:
            print(f"✅ Extracted: {result['provider']}, {result['year']} {result['quarter']}")
            records.append(result)
        else:
            print(f"❌ Failed to extract data from: {os.path.basename(pdf_file)}")
            failed_files.append(os.path.basename(pdf_file))

    print(f"\nTotal successfully extracted: {len(records)}")
    if failed_files:
        print(f"Total failed: {len(failed_files)}")
        print("Failed files:", failed_files)

📄 Processing: Q1 2024.pdf


Cannot set gray non-stroke color because /'P54' is an invalid float value
Cannot set gray non-stroke color because /'P56' is an invalid float value
Cannot set gray non-stroke color because /'P57' is an invalid float value
Cannot set gray non-stroke color because /'P59' is an invalid float value
Cannot set gray non-stroke color because /'P61' is an invalid float value
Cannot set gray non-stroke color because /'P62' is an invalid float value
Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P64' is an invalid float value
Cannot set gray non-stroke color because /'P65' is an invalid float value
Cannot set gray non-stroke color because /'P66' is an invalid float value
Cannot set gray non-stroke color because /'P68' is an invalid float value
Cannot set gray non-stroke color because /'P69' is an invalid float value
Cannot set gray non-stroke color because /'P70' is an invalid float value
Cannot set gray non-stroke color becau

❌ Failed to extract data from: Q1 2024.pdf
📄 Processing: Q1 2025.pdf


In [ ]:
# --- 4. DATAFRAME CREATION AND SAVING ---
if records:
    df = pd.DataFrame(records)
    column_order = [
        'provider', 'year', 'quarter', 
        'health', 'personal_accident', 'travel', 'total', 'source_file'
    ]
    df = df[column_order]

    out_csv = os.path.join(OUT_DIR, "premium_summary_universal.csv")
    df.to_csv(out_csv, index=False)
    
    print(f"\n✅ Data successfully saved to CSV: {out_csv}")
    print("\n--- DataFrame Preview ---")
    display(df) # 'display(df)' gives a nice formatted table in notebooks
else:
    print("\nNo data was extracted, so no file was saved.")
